First grab the information we want to use

In [100]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

driver = webdriver.Chrome()

input = "https://www.google.com/travel/things-to-do/see-all?g2lb=2502548,4208993,4254308,4258168,4260007,4270442,4271060,4274032,4291318,4305595,4306835,4308216,4314846,4317915,4328159,4329288,4330113,4338438,4270859,4284970,4291517,4292955,4316256&hl=en&gl=us&un=1&otf=1&dest_mid=/m/080h2&dest_state_type=sattd&sa=X&ved=0ahUKEwil66TDpcXnAhXUlXIEHczFDl8Qx2gIGA"
driver.get(input)
content = driver.page_source
soup = BeautifulSoup(content, 'html5lib')
#print(soup.prettify())
driver.close()

names = []
descriptions = []
ratings = []
reviews = []

for div in soup.findAll('div', attrs = {'class': 'GwjAi'}):
    try:
        name = div.find('div', attrs = {'class': 'skFvHc YmWhbc'}).get_text()
        description = div.find('div', attrs = {'class': 'nFoFM'}).get_text()
        rating = div.find('span', attrs = {'class':'KFi5wf'})
        review = div.find('span', attrs = {'class':'jdzyld'})
        
    except:
        break
        
    names.append(name)
    descriptions.append(description)
    ratings.append(rating.text if rating else "N/A")
    reviews.append(review.text if review else "N/A")
    
    
df = pd.DataFrame({'Attractions':names,'Descriptions':descriptions, 'Ratings': ratings, 'Reviews': reviews})
#print(df)

In [101]:
df.head()

Attractions                              Descriptions  \
0                Stanley Park  Sprawling urban park with scenic seawall   
1  Capilano Suspension Bridge   Landmark bridge perched in the treetops   
2            Granville Island                                      Beer   
3                     Gastown  Steam Clock, souvenirs & trendy eateries   
4                Canada Place   Iconic convention centre/ferry terminal   

  Ratings    Reviews  
0     4.8   (29,925)  
1     4.5   (18,158)  
2     4.6      (845)  
3     N/A        N/A  
4     4.5   (10,739)

From above, we see that in the 'Reviews' column, there are parentheses. Let's perform regex and remove them so that they are more readable.

In [102]:
# Convert using map and lambda if possible

clear_lst = []

for i in range(len(df)):
    if df.Reviews[i] == 'N/A':
        clear_lst.append('N/A')
    else:
        result = re.search(r'\((.*?)\)',df['Reviews'][i]).group(1)
        clear_lst.append(result)
        
df.Reviews = clear_lst
df

Attractions  \
0                        Stanley Park   
1          Capilano Suspension Bridge   
2                    Granville Island   
3                             Gastown   
4                        Canada Place   
..                                ...   
94              Burnaby Mountain Park   
95      Kids Market, Granville Island   
96              CRAB Park at Portside   
97  Canada Place Cruise Ship Terminal   
98  BC Sports Hall Of Fame and Museum   

                                Descriptions Ratings Reviews  
0   Sprawling urban park with scenic seawall     4.8  29,925  
1    Landmark bridge perched in the treetops     4.5  18,158  
2                                       Beer     4.6     845  
3   Steam Clock, souvenirs & trendy eateries     N/A     N/A  
4    Iconic convention centre/ferry terminal     4.5  10,739  
..                                       ...     ...     ...  
94  Picnicking, hiking, a playground & views     4.7   1,142  
95  Mall with children's shops & a play area     4.3   1,465  
96      Scenic green space with harbor views     4.1   1,190  
97                           Cruise terminal     4.4     586  
98        Museum of local athletic champions     4.3     103  

[99 rows x 4 columns]